# Evolving sorting networks through coevolution

In this lab session we are going to leverage coevolution to evolve **sorting networks**. Sorting networks are algorithms designed to arrange a sequence of elements into a specific order, usually ascending or descending. The key feature of sorting networks is that they use a fixed set of comparisons and swaps to achieve the sorting, and this set of operations is independent of the input data ([wikipedia](https://en.wikipedia.org/wiki/Sorting_network)).

We will represent networks as lists of comparators, where each comparator is a pair of indices indicating which elements to compare.

First of all, we import the random module and set the seed.

In [ ]:
import random
random.seed(0)

Let us define a function which sorts an array with a given sorting network.

In [ ]:
def eval_sorting_network(sn, array):
    # CODE HERE
    for s in sn:
        t = array[s[0]]
        if array[s[0]] > array[s[1]]:
            array[s[0]] = array[s[1]]
            array[s[1]] = t
    return array

In [ ]:
sn=[[0,2], [1,3], [0,3] ,[1,2], [0,1], [2,3]]
eval_sorting_network(sn, [6,4,2,5])

Now, we write the code to initialize the 2 competing populations: one for the networks and one for the arrays.

In [ ]:
def get_random_network(dim, depth):
    return [sorted(random.sample(dim, k=2)) for _ in range(0, depth)]

In [ ]:
def init_array_population(value_range, dim, pop_size):
    #CODE HERE
    pop = []
    for _ in range(0, pop_size):
        arr = []
        for _ in range(0, dim):
            arr.append(random.randint(0, value_range))
        pop.append(arr)
    return pop

def init_network_population(dim, min_depth, max_depth, pop_size):
    #CODE HERE
    pop = []
    for _ in range(0, pop_size):
        depth = random.randint(min_depth, max_depth)
        sn = []
        for _ in range(0, depth):
            sn.append(random.randint(0, dim - 1))
            sn.append(random.randint(sn[0] + 1, dim))
        pop.append(sn)
    return pop

Let us define the 2 fitness functions. A network will have good fitness if it can sort many arrays, while an array will have good fitness if it can 'trick' many networks.

In [ ]:
def net_fitness(net_pop, arr_pop, k=10):
    net_scores = [0]*len(net_pop)
    #CODE HERE
    c = 0
    for net in net_pop:
        fit = 0
        for arr in arr_pop:
            if eval_sorting_network(net, arr) == arr.sorted():
                fit += 1
        net_scores[c] = fit
        k -= 1
        c += 1
        if k == 0: break
    return net_scores

def arr_fitness(net_pop, arr_pop, k=10):
    arr_scores = [0]*len(arr_pop)
    #CODE HERE
    c = 0
    for arr in arr_pop:
        fit = 0
        for net in net_pop:
            if eval_sorting_network(net, arr) != arr.sorted():
                fit += 1
        arr_scores[c] = fit
        k -= 1
        c += 1
        if k == 0: break
    return arr_scores

We now implement the tournament selection and the crossover. We can use the same functions for both the populations.

In [ ]:
def tournament_selection(pop, scores, k):
  tournament = random.choices(range(len(pop)), k=k)
  #CODE HERE
  return selected[1]

In [ ]:
def one_point_crossover(x, y):
  #CODE HERE
  return of1#, of2

We can now implement a mutation function for the arrays.

In [ ]:
def array_mutation(x, value_range, p_m):
  def mutate(v):
    if random.random() < p_m:
        res = random.choice(value_range)
        while res == v: # we avoid sampling the same value
            res = random.choice(value_range)
        return res
    else:
      return v
  return [mutate(v) for v in x]

Now, we can choose one or more mutation operators for the sorting networks. You can use different ones during the evolution.

In [ ]:
def net_mutation(sn, value_range, p_m):
    #CODE HERE
    return sn

We have now all the elements to write the code for a generation.

In [ ]:
def get_best(pop, scores):
  return max(list(zip(scores, pop)))

In [ ]:
def generation(net_pop, arr_pop, net_scores, arr_scores, crossover, arr_dim, value_range, p_m, t_size):
  pop_size = len(net_pop)
  # perform selection for both the populations
  #CODE HERE
  # perform crossover
  #CODE HERE
  # apply the mutation operator(s) to the offspring
  #CODE HERE
  return net_pop, arr_pop

We can now define our `coevolution` function.

In [ ]:
def coevolution(value_range, 
                pop_size,
                arr_dim,
                min_depth,
                max_depth,
                net_fit,
                arr_fit,
                crossover,
                t_size = 10, 
                n_gen = 200,
                k_fit = 50):
  
  p_m = 1/arr_dim
  # initialize the population
  #CODE HERE
  net_scores = net_fit(Pt, Qt, k_fit)
  arr_scores = arr_fit(Pt, Qt, k_fit)
  net_history = [get_best(Pt, net_scores)[0]]
  arr_history = [get_best(Qt, arr_scores)[0]]
  
  for _ in range(0, n_gen):
    Ptm1 = Pt
    Qtm1 = Qt
    Pt, Qt = generation(Ptm1, Qtm1, net_scores, arr_scores, crossover, arr_dim, value_range, p_m, t_size)
    # evaluate the fitness
    #CODE HERE
    net_history.append(get_best(Pt, net_scores)[0])
    arr_history.append(get_best(Qt, arr_scores)[0])
    
  return get_best(Pt, net_scores)[1], get_best(Qt, arr_scores)[1], net_history, arr_history

Try your code for different array dimensions and parameters.

In [ ]:
best_net, best_arr, net_history, arr_history = coevolution(
    value_range=range(50), 
    pop_size=500,
    arr_dim=10,
    min_depth=15,
    max_depth=50,
    net_fit=net_fitness,
    arr_fit=arr_fitness,
    crossover=one_point_crossover,
    t_size = 10,
    n_gen = 200,
    k_fit = 50
    )

In [ ]:
print(best_net)
print(best_arr)

In [ ]:
random_arr = random.sample(range(50), k = 10)
print(random_arr)
eval_sorting_network(best_net, random_arr)

In [ ]:
eval_sorting_network(best_net, best_arr)

Plot the evolution of the fitness score of the best individual for both populations

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(net_history)
plt.ylabel('Network fitness')
plt.xlabel('Generation')
plt.show()

In [ ]:
plt.plot(arr_history)
plt.ylabel('Array fitness')
plt.xlabel('Generation')
plt.show()